In [8]:
pip install chromadb InstructorEmbedding

Note: you may need to restart the kernel to use updated packages.


In [13]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings, HuggingFaceInstructEmbeddings

In [14]:
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

loaders = []
loaders.append(PyPDFLoader("CIDS-OntoBeSS-v1.pdf").load()[0])
# loaders.append(PyPDFLoader("Common-Impact-Data-Standard-V1.1.pdf").load()[0])
# loaders.append(TextLoader('test.txt', encoding='utf8'))

loaders.append(DirectoryLoader('./test_scraped/', glob="./*.txt", loader_cls=TextLoader).load()[0])

In [14]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from chromadb.config import Settings

# Define the Chroma settings
PERSIST_DIRECTORY = 'DB'
CHROMA_SETTINGS = Settings(
        chroma_db_impl='duckdb+parquet',
        persist_directory=PERSIST_DIRECTORY,
        anonymized_telemetry=False
)


# text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(loaders)
embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl", model_kwargs={"device": 'cpu'})
db = Chroma.from_documents(texts, embeddings, persist_directory=PERSIST_DIRECTORY, client_settings=CHROMA_SETTINGS)
db.persist()
db = None

NameError: name 'loaders' is not defined

In [13]:
from langchain.llms import HuggingFacePipeline
from langchain.vectorstores import Chroma

# from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline


def load_model():
    '''
    Select a model on huggingface. 
    If you are running this for the first time, it will download a model for you. 
    subsequent runs will use the model from the disk. 
    '''
    model_id = "TheBloke/vicuna-7B-1.1-HF"
    tokenizer = LlamaTokenizer.from_pretrained(model_id)

    model = LlamaForCausalLM.from_pretrained(model_id,
                                            #   load_in_8bit=True, # set these options if your GPU supports them!
                                            #   device_map=1#'auto',
                                            #   torch_dtype=torch.float16,
                                            #   low_cpu_mem_usage=True
                                              )

    pipe = pipeline(
        "text-generation",
        model=model, 
        tokenizer=tokenizer, 
        max_length=2048,
        temperature=0,
        top_p=0.95,
        repetition_penalty=1.15
    )

    local_llm = HuggingFacePipeline(pipeline=pipe)

    return local_llm

db = Chroma(persist_directory='DB', embedding_function=embeddings, client_settings=CHROMA_SETTINGS)
retriever = db.as_retriever()

# load the LLM for generating Natural Language responses. 
llm = Cohere(cohere_api_key=COHERE_API_KEY)
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)

NameError: name 'embeddings' is not defined

In [40]:
query = "What is an Organization according to the Common Impact Data Standard?"
res = qa(query)

# query = "According to the Common Impact Data Standard ontology (an ontology for representing impact), find the following for Edmonton sexual assault DNA phenotyping story: Organization, Impact Model, Program, Service, Activity, Input, Output, Outcome, Stakeholder, Stakeholder Outcome, Impact Report, Impact Risk, Indicator, Indicator Report, Impact Scale, Impact Depth, Impact Duration"

# query = "what organization is involved in Edmonton sexual assault DNA phenotyping"
# query = "what is the impact scale in Edmonton sexual assault DNA phenotyping"
# index.query(query)

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 8.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 10.0 seconds as it raised RateLimitError: You exceeded your current quota, please ch

RateLimitError: You exceeded your current quota, please check your plan and billing details.

In [12]:
query = "What did the president say about Ketanji Brown Jackson"
index.query_with_sources(query)

{'question': 'What did the president say about Ketanji Brown Jackson',
 'answer': " The president said that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson, one of the nation's top legal minds, to continue Justice Breyer's legacy of excellence, and that she has received a broad range of support from the Fraternal Order of Police to former judges appointed by Democrats and Republicans.\n",
 'sources': 'state_of_the_union.txt'}

In [13]:
index.vectorstore

In [14]:
index.vectorstore.as_retriever()

VectorStoreRetriever(vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x1069ff2b0>, search_type='similarity', search_kwargs={})

In [50]:
article = open("test.txt").read()
article

'Green books are of type 1. Red books are of type 2. Yellow books are of type 3.\n\nJohn has a green book.'

In [2]:
COHERE_API_KEY = "Q4JBi3f5v6xjKgkFns4Az10PfaCskkbk1wi4Bb6f"
from langchain.llms import Cohere
from langchain import PromptTemplate, LLMChain

In [44]:
template = """Question: {question}

Common impact data standard (CIDS) ontology is defined as follows:
Organization: A group of people working together to achieve a common goal. 

Impact Model: A framework for understanding and measuring the impact of an organization's work. 

Program: A set of activities that are designed to achieve a specific outcome. 

Service: A product or service that is provided to a customer or client. 

Activity: A specific task that is performed as part of a program or service. 

Input: A resource that is used to produce an output. 

Output: A product or service that is produced as a result of an activity. 

Outcome: A change in the state of a system that is caused by an output. 

Stakeholder: A person or group who has an interest in the outcome of an organization's work. 

Stakeholder Outcome: A change in the state of a stakeholder that is caused by an organization's work. 

Impact Report: A document that describes the impact of an organization's work. 

Impact Risk: A factor that could potentially reduce the impact of an organization's work. 

Indicator: A measurable variable that is used to track the progress of an organization's work. 

Indicator Report: A document that describes the results of an organization's impact measurement efforts. 

Impact Scale: The size of the impact of an organization's work. 

Impact Depth: The depth of the impact of an organization's work. 

Impact Duration: The length of time that an organization's impact is felt. 
 
Answer: let's think step by step"""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [45]:
llm = Cohere(cohere_api_key=COHERE_API_KEY)

In [46]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [48]:
# question = "What is the organization according to Common impact data standard in the article at https://www.aiaaic.org//aiaaic-repository/ai-and-algorithmic-incidents-and-controversies/ndis-independent-assessments-robo-planning?"
# question = "find stakeholders in the article at https://www.aiaaic.org//aiaaic-repository/ai-and-algorithmic-incidents-and-controversies/ndis-independent-assessments-robo-planning?"
question = "According to CIDS ontology, what is the impact duration in the article at https://www.aiaaic.org//aiaaic-repository/ai-and-algorithmic-incidents-and-controversies/gm-chevrolet-bolt-motorbike-collision?"

llm_chain.run(question)

'\nThe impact duration in the article at https://www.aiaaic.org//aiaaic-repository/ai-and-algorithmic-incidents-and-controversies/gm-chevrolet-bolt-motorbike-collision/ is "the length of time that an organization\'s impact is felt."\nSo the answer is Impact Duration.'